# QuantAgent 演示：股票数据获取与可视化

本 Notebook 旨在演示如何通过 Tushare API 获取指定股票的 OHLCV 数据，并生成交互式图表和统计表格。这些输出可以直接截图或导出，用于 PowerPoint 展示。

In [1]:
# 1. 导入必要的库
import sys
import os

# 将项目根目录添加到路径，以便导入配置
sys.path.append(os.path.abspath(".."))

import tushare as ts
import pandas as pd
import mplfinance as mpf
from config.settings import TUSHARE_TOKEN

print("Libraries imported successfully.")

Libraries imported successfully.


In [2]:
# 2. 初始化 Tushare Pro 接口
# 使用配置文件中的 Token
ts.set_token(TUSHARE_TOKEN)
pro = ts.pro_api()

print(f"Tushare initialized with token: {TUSHARE_TOKEN[:6]}******")

Tushare initialized with token: d47f91******


In [3]:
# 3. 定义获取股票数据的函数
def get_stock_ohlcv(ts_code, start_date, end_date):
    """
    获取指定股票的日线行情数据
    :param ts_code: 股票代码 (e.g., '600519.SH')
    :param start_date: 开始日期 (YYYYMMDD)
    :param end_date: 结束日期 (YYYYMMDD)
    :return: DataFrame with OHLCV data
    """
    df = pro.daily(ts_code=ts_code, start_date=start_date, end_date=end_date)
    if df.empty:
        print(f"No data found for {ts_code}")
        return None
    
    # 数据预处理
    df = df.sort_values('trade_date')
    df['trade_date'] = pd.to_datetime(df['trade_date'])
    df.set_index('trade_date', inplace=True)
    
    # 重命名列以符合 mplfinance 要求
    df = df.rename(columns={
        'vol': 'volume'
    })
    
    return df[['open', 'high', 'low', 'close', 'volume']]

print("Function defined.")

Function defined.


In [4]:
# 4. 获取指定股票的 OHLCV 数据
# 示例：贵州茅台 (600519.SH) 近半年的数据
ts_code = '600519.SH'
start_date = '20240601'
end_date = '20241201'

df = get_stock_ohlcv(ts_code, start_date, end_date)

if df is not None:
    print(f"Data retrieved for {ts_code}. Shape: {df.shape}")
    display(df.head())
    display(df.tail())

Data retrieved for 600519.SH. Shape: (122, 5)


,open,high,low,close,volume
trade_date,,,,,
2024-06-03,1650.00,1650.92,1635.11,1639.39,30261.05
2024-06-04,1640.50,1656.00,1636.55,1653.20,20841.08
2024-06-05,1652.00,1658.68,1643.03,1644.00,19055.57
2024-06-06,1643.66,1651.99,1635.91,1639.81,25610.05
2024-06-07,1646.13,1646.18,1619.01,1619.18,33524.55


,open,high,low,close,volume
trade_date,,,,,
2024-11-25,1511.00,1532.00,1496.67,1498.57,33185.56
2024-11-26,1498.57,1518.88,1488.88,1509.00,25744.79
2024-11-27,1509.00,1522.15,1505.00,1519.05,25934.17
2024-11-28,1522.00,1524.99,1502.00,1518.00,23952.03
2024-11-29,1517.00,1555.00,1513.21,1525.74,36088.10


In [ ]:
# 5. 绘制 K 线图 (Candlestick Chart)
# 使用 mplfinance 绘制专业的金融图表
if df is not None:
    # 设置图表样式
    mc = mpf.make_marketcolors(up='r', down='g', edge='i', wick='i', volume='in', inherit=True)
    s = mpf.make_mpf_style(marketcolors=mc)

    # 绘制图表
    mpf.plot(df, 
             type='candle', 
             mav=(5, 10, 20), 
             volume=True, 
             title=f'\n{ts_code} OHLCV Chart ({start_date}-{end_date})',
             style=s,
             figsize=(12, 8))
else:
    print("No data to plot.")

In [ ]:
# 6. 生成关键统计指标表格
if df is not None:
    stats = {
        "指标": ["最高价", "最低价", "平均收盘价", "总成交量 (手)", "价格波动率 (std)"],
        "数值": [
            df['high'].max(),
            df['low'].min(),
            round(df['close'].mean(), 2),
            int(df['volume'].sum()),
            round(df['close'].std(), 2)
        ]
    }
    stats_df = pd.DataFrame(stats)
    display(stats_df)
else:
    print("No data for statistics.")